In the class we try to predict the restaurant rating using the dot product. One thing we found was that it predict a lot of non-sense rating like negative number or 6. In this homework, we are going to fix this problem

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

The problem with dot product is that it is unbounded. So we need to bound it between 0 and 5. The most common way to do this is to use logistic function to turn $(-\infty, \infty)$ to a bounded region.
$$ \theta(s) = \frac{1}{1 + e^{-s}}$$

1) Given the restaurant attribute $\vec{\rho}^{(r)}$ and person preference $\vec{\pi}^{(p)}$. Write down the prediction formula which gives the output in the range of $(0,5)$.

Hint: use dot product and logistic function then scale it properly.

In [2]:
def scaler(x):
    return (1./ (1 + np.exp(-x))) * 5

def guess(R, P):
    return scaler(np.dot(R.T, P))

2) Write down the cost function with your prediction formula above.

In [4]:
def score(R, P):
    return np.sum(H*(T - guess(R,P))**2) # cost function

3) (Optional) Show that if your predition formula is (Do it on paper by hand. It's actually just a chain rule.)
$$Guess_{r,p} = a \theta(\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}) + d$$



Then the derivative is given by
$$
	\frac{\partial{c}}{\partial{\pi^{(p)}_i}} =
	\sum_r 2 h_{rp} \left[ a \cdot \frac{1}{1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)} } }  + d - T_{rp} \right] \cdot \frac{ a e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} }{\left( 1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} \right)^2} \rho^{(r)}_i
$$
and
$$
	\frac{\partial{c}}{\partial{\rho^{(r)}_i}} =
	\sum_p 2 h_{rp} \left[ a \cdot \frac{1}{1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)} } }  + d - T_{rp} \right] \cdot \frac{ a e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} }{\left( 1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} \right)^2} \pi^{(p)}_i
$$



4) Write the above two equations in matrix form given that matrix

$$
    S_{rp} = 2 h \otimes \left[ a \cdot \frac{1}{1 + e^{ -R^T P } }  + d - T \right] \cdot \frac{ a e^{ -R^T P} }{\left( 1 + e^{ R^T P} \right)^2}
$$
where the exponential is element-wise exponential(yes there is such thing as exponential of matrix but that's not what we want).

The partial derivative should look super simple in terms of $S$.

In [5]:
# dcost /dP = R x HT
# dcost /dR = P x HT.T

5) Write down the update rule for R and P. Use $a$ and $d$ you found in 1.

$P = P - \eta\; dP$

$P_{new} = P_{old} + 2\lambda R_{old} (H \otimes (\tilde{T} - thetha (R^T_{old}P_{old})))$


$R = R- \eta\; dR$

$R_{new} = R_{old} + 2\lambda P_{old} (H \otimes (\tilde{T} - thetha (R^T_{old}P_{old})))^T$

$thetha = 1./ (1 + np.exp(-x))) * 5 $

In [46]:
def theta(s):
    return (1./(1 + np.exp(-s)))
a = 5 # scaler 
l = 0.001
def find_R_P(R, P):
    for i in range(100000):
        RP = np.dot(R.T, P)
        HT = (2*H*l)  *  ((a*theta(RP)) + d -T)  *  ((a*np.exp(-RP))/(1 + np.exp(-RP))**2) # S(R,P)
        
        R = R - np.dot(P, HT.T)
        P = P - np.dot(R, HT)
        
        if (i%5000 == 0):
            print i, score(R, P)
    return R, P

6) Given the rating matrix we use in class, use this new prediction function and update rule to find $R$ and $P$.

In [6]:
def read_rating():
    with open('Exercise 10/rating.csv') as f:
        iline = 0
        lines = f.readlines()
        useful_lines = lines[3:]

        names = lines[2].split(',')[2:]
        names = map(lambda x: x.strip(), names)

        all_ratings = []
        all_defined = []
        rnames = []
        for iline, line in enumerate(useful_lines):
            tokens = line.split(',')
            tokens = map(lambda x: x.strip(), tokens)
            rname = tokens[1]
            ratings = tokens[2:]
            defined = map(lambda x: 0 if x=='' or x=='"' else 1, ratings )
            def clean_cast(x):
                # print x
                return 0 if x=='' or x=='"' else float(x)
            ratings = map(lambda x: clean_cast(x), ratings)
            all_ratings.append(ratings)
            all_defined.append(defined)
            rnames.append(rname)
        #print all_ratings, all_defined , rnames
        T = np.array(all_ratings)
        H = np.array(all_defined)
    return T, H, names, rnames
T, H, names, rnames = read_rating()

In [54]:
npeople = len(names)
nrest = len(rnames)
nfeature = 5
np.random.seed(17)
P = np.random.randn(nfeature, npeople)
R = np.random.randn(nfeature, nrest)
print P.shape
print R.shape

(5L, 44L)
(5L, 41L)


In [57]:
def theta(s):
    return (1./(1 + np.exp(-s)))
a = 5 # scaler 
l = 0.005
def find_R_P(R, P):
    for i in range(100000):
        RP = np.dot(R.T, P)
        HT = (2*H*l)  *  ((a*theta(RP))-T)  *  ((a*np.exp(-RP))/(1 + np.exp(-RP))**2) # S(R,P)
        
        R = R - l * np.dot(P, HT.T)
        P = P - l * np.dot(R, HT)
        
        if (i%5000 == 0):
            print i, score(R, P)
    return R, P

In [58]:
print score(R,P)
R,P = find_R_P(R,P)

580.869413997
0 580.771999737
5000 388.469213064
10000 319.076377315
15000 279.030300976
20000 252.999544919
25000 235.88184564
30000 224.04681705
35000 214.786552417
40000 207.25583545
45000 201.088010351
50000 195.98234637
55000 191.699463298
60000 188.06657773
65000 184.950739183
70000 182.235316551
75000 179.845028974
80000 177.782003653
85000 176.033112349
90000 174.53121778
95000 173.21628631


7) Use the code we had in exercise to show the prediction table.

In [37]:
from IPython.display import HTML

class TableCell:
    
    def __init__(self, text, tc=None, color=None):
        self.text = text
        self.tc = tc
        self.color = color
    
    def to_html(self):
        return '<td>%s</td>'%self.text

#the rating and guess matrix has different convention from the notes so be sure to transpose it first
def maketable(rating, has_rating, guess, restaurants, names):
    n_rests = len(restaurants)
    n_names = len(names)
    tab = np.empty((n_rests+1, n_names+1),dtype='object')
    #print tab.shape

    for irest in range(n_rests):
        tab[irest+1,0] = restaurants[irest]

    for iname in range(n_names):
        tab[0,iname+1] = names[iname]

    for irest in range(n_rests):
        for iname in range(n_names):
            if not has_rating[iname, irest]:
                tab[irest+1, iname+1] = TableCell('<span style="color:red">%3.2f</span>'%(guess[iname, irest]))
            else:
                tab[irest+1, iname+1] = TableCell('<span style="color:blue">%3.2f</span><span style="color:red">(%3.2f)</span>'%(rating[iname, irest], guess[iname, irest]))
    #now convert tab array to nice html table
    nrow, ncol = tab.shape
    t = []
    t.append('<table>')
    for irow in range(nrow):
        t.append('<tr>')
        for icol in range(ncol):
            cell = tab[irow,icol]
            if cell is not None:
                if isinstance(cell,TableCell):
                    t.append(tab[irow, icol].to_html())
                else:
                    t.append('<td>')
                    t.append(tab[irow, icol])
                    t.append('</td>')
            else:
                t.append('<td></td>')
        t.append('</tr>')  
    t.append('</table>')
    return '\n'.join(t)

In [38]:
# call it like this

HTML(maketable(T.T, H.T, guess(R, P).T, rnames, names))

,Piti,Meena,Pitoon,Sohum,Sam,Keng,O,Yok,Kitty,Time,Robroo,Peem,Chanon,Ohm,Sun,Tii,Tow,John,Mint,Opal,Kelly,Jay,Sharik,Ou,PJ,Punawit,Martin,Ploy,Majeed,Bossy,Sea,Pan,Ice,Karn,May,Rin,Peter,Ham,Benz,Billy,Kanat,Sam,Boss,Best
Mai-tok-mai-tak,0.51,2.60,2.00(1.98),3.16,4.99,2.00(2.77),2.00(2.52),3.00(2.93),2.69,3.00(3.22),3.00(3.20),5.00(4.16),2.44,3.00(3.27),2.00(2.10),3.00(3.17),3.00(3.02),2.65,2.00(0.70),3.00(2.60),3.00(3.13),3.00(2.67),3.60,2.00(2.53),0.58,2.17,2.57,4.00(3.10),3.06,3.00(3.42),4.00(3.45),3.94,1.40,3.00(3.52),4.00(4.22),2.82,2.45,3.15,4.00(3.76),4.00(4.43),4.00(3.46),2.98,4.00(3.97),5.00(4.09)
Puttharaksa,4.00(3.76),3.09,4.00(2.13),4.00(3.60),4.00(4.00),3.00(3.19),4.00(3.55),3.00(3.31),3.31,3.00(3.05),3.14,4.00(3.67),3.00(2.60),4.00(3.59),3.00(2.89),4.00(3.85),4.00(4.25),3.00(3.00),3.00(2.89),3.00(3.00),3.00(3.69),3.00(2.83),3.00(3.06),3.00(2.79),2.64,3.26,2.86,3.00(2.87),3.00(2.92),3.02,4.00(3.74),4.00(3.99),4.00(4.09),3.21,4.00(2.81),3.00(3.39),3.20,2.84,1.00(3.11),4.00(4.02),3.00(2.99),2.69,4.00(3.54),4.00(3.55)
Big Mamma,5.00,4.00(3.97),1.04,5.00(4.78),5.00(5.00),5.00,4.77,5.00,4.00(4.07),4.15,0.45,4.71,3.62,5.00,4.86,0.00,5.00,5.00,5.00,4.73,5.00,4.12,4.85,3.18,5.00,4.98,4.22,4.26,4.19,0.84,4.99,0.00,5.00,1.15,4.00(3.99),4.00(4.01),0.00,3.00(3.04),2.60,0.19,2.63,5.00,0.48,0.13
Seefah,0.09,2.00(1.79),4.00(3.98),4.00(4.41),5.00(4.84),1.93,4.00(3.92),3.00(3.18),4.64,5.00(4.49),0.04,4.00(3.91),3.00(2.94),0.00,0.11,4.00(3.92),0.00,3.00(3.03),0.00,0.43,0.00,1.59,4.00(3.91),2.00(2.07),3.51,4.00(3.99),2.67,3.00(3.29),4.06,2.00(1.99),0.01,5.00,0.00,4.65,3.00(3.09),0.00,3.00(3.02),4.00(3.96),4.99,4.00(5.00),4.95,0.00,5.00(4.94),4.00(4.98)
Music Square,2.00(2.01),4.43,0.00,2.82,3.00(3.00),4.00(3.93),4.96,4.00(4.01),4.00(3.92),4.00(3.81),4.00(5.00),5.00,0.17,5.00,4.00(3.92),4.00(4.00),5.00(5.00),2.02,5.00(4.91),4.78,4.00(3.96),3.00(3.03),4.33,2.00(2.52),3.10,5.00(4.85),1.92,3.00(3.56),4.73,4.99,4.00(5.00),0.00,5.00,4.00(3.79),5.00(5.00),4.00(5.00),0.02,0.03,3.87,2.00(1.95),4.00(4.14),5.00,4.00(4.03),3.00(3.02)
Mamma Mia,3.00(3.00),4.00(3.80),1.01,5.00(4.33),5.00(4.96),3.77,4.00(4.10),4.00(3.91),3.58,4.00(3.60),5.00(4.74),5.00(4.84),3.00(2.44),5.00(4.99),3.00(3.65),5.00(4.77),4.00(5.00),3.01,4.00(4.19),4.00(3.95),4.00(4.97),4.00(3.41),4.03,3.00(3.16),0.51,3.00(3.21),4.00(3.15),3.00(3.51),3.42,4.00(4.30),5.00(4.92),3.00(3.00),5.00(4.97),4.00(4.09),5.00(4.48),5.00(4.97),4.09,3.00(3.30),3.00(3.43),5.00(4.83),3.00(3.23),4.50,4.00(4.51),4.51
Srijan,4.00(4.04),2.76,3.00(2.39),3.60,5.00,3.00(3.67),3.47,5.00(3.77),3.41,3.00(3.30),2.00(1.62),4.00(3.60),2.71,2.00(1.81),2.00(2.36),2.00(2.05),2.00(2.19),3.96,1.00(1.28),2.60,1.70,2.67,3.40,3.00(2.59),3.73,4.00(3.65),2.86,2.99,3.00(3.15),2.00(2.43),2.79,4.00(3.99),1.75,3.07,3.00(3.04),0.94,0.99,3.00(2.90),3.00(3.78),4.00(3.88),3.00(3.53),2.31,4.00(3.38),3.00(3.39)
Steak House,0.00,3.00(3.26),0.09,4.00(4.06),4.00(4.01),1.94,4.30,2.88,4.00(4.02),4.46,4.99,5.00,1.00,4.95,0.72,4.87,4.92,1.00(0.99),0.00,2.79,0.00(0.11),2.52,5.00(4.63),3.00(2.39),0.03,3.00(2.99),2.20,3.92,4.50,4.93,4.96,3.00(3.00),0.04,4.81,5.00,5.00,3.00(3.02),2.00(2.02),4.95,5.00,4.87,4.90,4.97,4.98
Anya,1.00(0.98),4.00(4.30),4.00(3.99),4.00(4.93),5.00(4.78),4.00(3.86),4.00(3.73),4.00(4.20),4.00(4.02),4.00(4.22),3.00(2.94),3.00(2.80),4.00(3.98),1.00(1.14),4.00(4.15),5.00,4.77,2.00(2.29),4.00(3.89),4.00(4.22),4.00(5.00),5.00(4.18),5.00(4.61),4.00(3.96),0.00,1.24,4.00(4.00),4.00(4.18),3.00(3.01),4.00(4.31),4.66,5.00,5.00(5.00),4.86,0.63,0.02,5.00,5.00(4.92),4.00(3.96),5.00(5.00),3.00(3.34),0.16,5.00(4.98),4.00(4.99)
Chicky Chic,2.00(2.20),3.00(3.08),2.00(1.58),2.00(2.74),0.00(0.00),2.00(1.19),5.00(3.80),2.00(1.44),3.00(3.39),2.00(2.40),5.00(4.51),4.00(3.89),2.00(1.97),1.00(0.92),2.00(2.34),4.80,2.65,0.00(0.63),2.00(1.91),2.00(2.56),0.00(0.58),2.00(2.36),2.00(1.54),3.00(2.68),3.00(2.98),3.00(3.05),3.00(2.40),2.00(2.09),2.57,3.00(3.55),3.00(3.28),4.30,3.00(2.75),3.00(3.07),3.00(2.78),5.00(4.93),5.00(4.84),2.00(1.58),2.00(2.58